In [1]:
import os
import torch
import pydssp
import numpy as np
from utils.utils import *

In [2]:
'''
    H = α-helix
    B = residue in isolated β-bridge
    E = extended strand, participates in β ladder
    G = 310-helix
    I = π-helix
    P = κ-helix (poly-proline II helix)
    T = hydrogen-bonded turn
    S = bend
'''
aa = {'ALA':0, 'VAL':0, 'ILE':0, 'LEU':0, 'MET':0, 'PHE':0, 'TYR':0, 'TRP':0, 'CYS':0, 'GLY':0,
      'PRO':0, 'SER':0, 'THR':0, 'ASN':0, 'GLN':0, 'ARG':0, 'HIS':0, 'LYS':0, 'ASP':0, 'GLU':0}
aa_dict = {'A':aa.copy(), 'H':aa.copy(), 'B':aa.copy(), 'E':aa.copy(),
           'G':aa.copy(), 'I':aa.copy(), 'P':aa.copy(), 'T':aa.copy(), 'S':aa.copy()}
aa_dict = {0:aa.copy(), 1:aa.copy(), 2:aa.copy(), 'A':aa.copy()}

In [3]:
pdb_chain = np.load('mem_prot_rep.npy')

In [4]:
atomname_dict = {'N':0,'CA':1,'C':2,'O':3}
for ii,i in enumerate(pdb_chain):
    pdb = PDB(f'./PDB/{i[0]}.cif')
    chain_id = i[1].strip('\n')
    if not chain_id:
        chain_id = 'A'
    chain_idx = np.where(pdb.chain == chain_id)[0]
    res_num, res_index = np.unique(pdb.resnum[chain_idx], return_inverse=True)
    if len(res_num)<10:
        print(i[0])
        continue
    current_aa = np.empty((len(res_num)), dtype='<U3')
    coord = torch.zeros((len(res_num)),4,3)
    for i_idx, idx in enumerate(chain_idx):
        if pdb.atomname[idx] in ('N', 'CA', 'C', 'O'):
            coord[res_index[i_idx],atomname_dict[pdb.atomname[idx]]] = torch.Tensor(pdb.coords[idx])
            current_aa[res_index[i_idx]] = pdb.resname[chain_idx][i_idx]
    dssp = pydssp.assign(coord, out_type='index')
    for aa_i, amino_acid in enumerate(current_aa):
        if amino_acid not in aa.keys():
            continue
        aa_dict['A'][amino_acid] += 1
        aa_dict[int(dssp[aa_i])][amino_acid] += 1
    print(ii)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
5N9I
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
2VJ0
177
3NVE
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
2VJ0
201
3IIQ
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
3NVF
228
229
230
231
232
233
234
235
236
237
238
239
240
7P8X
242
243
244
245
2Y3J
247
248
249
250
251
252
253
254
255
256
257
258
2FGR
260
261
262
263
264
265
266
267
268
269
270
3JQL
272
273
274


In [6]:
aa_dict

{0: {'ALA': 3679,
  'VAL': 2607,
  'ILE': 2018,
  'LEU': 3840,
  'MET': 927,
  'PHE': 2096,
  'TYR': 1768,
  'TRP': 809,
  'CYS': 919,
  'GLY': 7113,
  'PRO': 4741,
  'SER': 4562,
  'THR': 3663,
  'ASN': 3912,
  'GLN': 2020,
  'ARG': 2461,
  'HIS': 1398,
  'LYS': 3300,
  'ASP': 4528,
  'GLU': 2837},
 1: {'ALA': 7979,
  'VAL': 5233,
  'ILE': 5082,
  'LEU': 9170,
  'MET': 1880,
  'PHE': 4073,
  'TYR': 2710,
  'TRP': 1461,
  'CYS': 983,
  'GLY': 3409,
  'PRO': 1901,
  'SER': 3713,
  'THR': 3383,
  'ASN': 2361,
  'GLN': 2834,
  'ARG': 3484,
  'HIS': 1351,
  'LYS': 3597,
  'ASP': 2796,
  'GLU': 4085},
 2: {'ALA': 2165,
  'VAL': 3626,
  'ILE': 2522,
  'LEU': 3121,
  'MET': 671,
  'PHE': 1820,
  'TYR': 1973,
  'TRP': 734,
  'CYS': 576,
  'GLY': 1841,
  'PRO': 670,
  'SER': 1964,
  'THR': 2418,
  'ASN': 1060,
  'GLN': 1163,
  'ARG': 1565,
  'HIS': 715,
  'LYS': 1542,
  'ASP': 1181,
  'GLU': 1381},
 'A': {'ALA': 13823,
  'VAL': 11466,
  'ILE': 9622,
  'LEU': 16131,
  'MET': 3478,
  'PHE': 7989,

In [22]:
pdb = PDB(f'./PDB/{i[0]}.cif')

In [17]:
test = 'ATOM   12   N  N     . ASN A  1 2  ? 30.943  13.260  40.012  1.00   16.27  ? 2   ASN A N     1 '

In [21]:
test.split()

['ATOM',
 '12',
 'N',
 'N',
 '.',
 'ASN',
 'A',
 '1',
 '2',
 '?',
 '30.943',
 '13.260',
 '40.012',
 '1.00',
 '16.27',
 '?',
 '2',
 'ASN',
 'A',
 'N',
 '1']